In [55]:
import pickle
import numpy as np
import pandas as pd
import os
import zipfile
import shutil
from datetime import datetime,timedelta
from scipy.stats import iqr,skew,kurtosis
from joblib import Parallel,delayed
import pickle
import numpy as np
import pandas as pd
import os
import zipfile
import shutil
from datetime import datetime,timedelta
from scipy.stats import iqr,skew,kurtosis
from joblib import Parallel,delayed
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.decomposition import PCA
from pprint import pprint
from sklearn.metrics import f1_score,r2_score,classification_report
from sklearn.model_selection import ParameterGrid
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,RandomForestRegressor
from sklearn.svm import SVC
# print(m/(m+n),'baseline accuracy')
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix,f1_score,precision_score,recall_score,accuracy_score
import itertools
from sklearn.model_selection import ParameterGrid, cross_val_predict, GroupKFold,GridSearchCV,StratifiedKFold
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt

In [56]:
file_directory = './data/parsed_data/'
save_directory = './data/lagged_data/'

In [57]:
os.listdir(file_directory)

['.ipynb_checkpoints',
 'with_non_lapsers_observation_10_prediction_60.p',
 'with_non_lapsers_observation_20_prediction_60.p',
 'with_non_lapsers_observation_30_prediction_60.p',
 'with_non_lapsers_observation_45_prediction_60.p',
 'with_non_lapsers_observation_60_prediction_60.p']

In [58]:
def get_feature_sequence(df,K=5):
    df  = df.sort_values('time').reset_index(drop=True)
    df['feature_final'] = df['feature_final'].apply(lambda a:np.float64(a.reshape(-1)[2:].reshape(-1,1)))
    time_of_day = df['Time of Day'].values[0]
    age = df['Gender'].values[0]
    user_id = df.user.values[0]
    all_data = []
    for i in range(K-1,df.shape[0],1):
        row = df.iloc[i]
        features = np.concatenate(list(df['feature_final'].iloc[i-K+1:i+1])).reshape(1,K,-1)
        label = df['Label'].iloc[i]
        day = df['day'].iloc[i]
        time = df['time'].iloc[i]
        start_time = df['start_time'].iloc[i-K+1]
        end_time = df['end_time'].iloc[i]
        all_data.append([features,user_id,label,day,time,time_of_day,age,start_time,end_time])
    return pd.DataFrame(all_data,columns=['features','user','label','day','time','Time of Day','Gender','start_time','end_time'])
Ks = np.arange(2,30,2)
for f in os.listdir(file_directory):
    if f[:3]=='lag' or f[0]=='.':
        continue
    obs = f.split('_')[4]
    prediction = f.split('_')[-1].split('.')[0]
    if os.path.isdir(save_directory+'obs_'+str(obs)+'_prediction_'+str(prediction)):
        continue
    if not os.path.isdir(save_directory+'obs_'+str(obs)+'_prediction_'+str(prediction)):
        os.makedirs(save_directory+'obs_'+str(obs)+'_prediction_'+str(prediction))
    print(f)
    saving_directory = save_directory+'obs_'+str(obs)+'_prediction_'+str(prediction)+'/'
    data = pickle.load(open(file_directory+f,'rb'))
    for K in Ks:
        df_seq = pd.concat([get_feature_sequence(df,K=K) for i,df in data.groupby('user',as_index=False)])
        pickle.dump(df_seq,open(saving_directory+'lagged_'+str(K)+'.p','wb'))
    print('done',f)

with_non_lapsers_observation_45_prediction_60.p
done with_non_lapsers_observation_45_prediction_60.p
with_non_lapsers_observation_60_prediction_60.p
done with_non_lapsers_observation_60_prediction_60.p


In [52]:
saving_directory

'./data/lagged_data/obs_30_prediction_60'

In [59]:
data  = pickle.load(open('./data/lagged_data/obs_60_prediction_60/lagged_28.p','rb'))

X = np.concatenate(list(data['features']))
X1 = X.reshape(X.shape[0],-1)
y = data['label'].values
y = np.int64(np.array(y))
y[y>0] = 1
y[y<1] = -1
groups = data['user'].values

X_time = data['Time of Day'].values.reshape(-1,1)

from sklearn import preprocessing
X_time = preprocessing.OneHotEncoder().fit_transform(X_time).todense()

X_gender = data.Gender.values.reshape(-1,1)

X_gender = preprocessing.OneHotEncoder().fit_transform(X_gender).todense()

X = np.concatenate([X_time,X_gender,X1],axis=1)

X.shape,y.shape,groups.shape

In [94]:
1808-28*64

16

In [86]:
def convert_to_lapsers_only(df):
    def helper(df):
        if df.Label.sum()==0:
            return pd.DataFrame([],columns=list(df.columns))
        return df
    return df.groupby('user',as_index=False).apply(helper)

In [87]:
data  = pickle.load(open('./data/parsed_data/with_non_lapsers_observation_60_prediction_60.p','rb'))
data= convert_to_lapsers_only(data)

In [91]:
data.Label.

961